In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. 뉴스 기사 복사-붙여넣기
text = """
2025학년도 대학수학능력시험(수능) 가채점 기준 고득점자들의 윤곽이 드러나면서 지난해 1명뿐이던 수능 만점자가 이번에는 대폭 늘어날 것으로 전망된다.

27일 교육계에 따르면 수능 가채점 결과 서울 서초구 세화고에서 3학년 A군과 같은 학교 졸업생 B씨가 각각 만점을 받은 것으로 파악됐다. A군은 이미 고려대와 성균관대 의예과 등의 수시모집에 지원했으며 합격 결과를 기다리는 것으로 전해졌다. B씨는 고려대 공과대학 재학생인 것으로 알려졌다.

올해 수능은 지난해보다 다소 쉽게 출제돼 만점자와 1∼2개만 틀린 초고득점 학생이 급증한 것으로 예상된다. 특히 만점자는 작년(1명)보다 급증한 10명 안팎이 될 것이라는 전망도 나온다.

남윤곤 메가스터디 입시전략연구소장은 “올해 수능 만점자는 재학생 4명, 재수생 4명으로 파악되는데 (세화고 졸업생) B씨는 포함되지 않았다”며 “가채점 결과가 확실하다면 만점자는 최소 9명”이라고 말했다.

서울대·연세대 의대의 경우 최대 1문제를 틀려야 합격권에 들 수 있다는 분석도 나온다. 서울중등진학지도연구회에 따르면 올해 수능 가채점 원점수(300점 만점) 기준 서울대·연세대 의예과 합격권은 297점으로 예상됐다. 고려대·성균관대·울산대·가톨릭대 의예과는 296점, 수도권 의대는 293점으로 예측했다.

장지환 서울중등진학지도연구회 교사는 “작년 수능보다 다소 쉬워 수도권 의대는 2∼3개 정도를 틀려야 합격권에 들 수 있을 것으로 추정된다”고 말했다. 이어 “서울대 의대는 과목 선택을 어떻게 하느냐가 관건”이라며 “2과목을 선택하거나 1과목에서는 물리, 화학보다 지구과학이나 생물을 선택한 학생이 표준점수에서 유리할 것으로 보인다”고 덧붙였다.
"""

# 문장 분리
sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]

# TF-IDF 계산
stop_words = ["그리고", "또는", "하지만", "있는", "같은", "이다", "한다", "있다"]
vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(sentences)
terms = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray().sum(axis=0)

# 상위 키워드 추출 (상위 7개)
top_n = 7
top_keywords = [terms[i] for i in tfidf_scores.argsort()[-top_n:]]
print("TF-IDF 기반 키워드:", top_keywords)

# 문장별 가중치 계산 (TF-IDF 점수 합산)
weighted_sentences = {}
for i, sentence in enumerate(sentences):
    weight = sum(
        tfidf_scores[terms.tolist().index(keyword)] if keyword in sentence else 0
        for keyword in top_keywords
    )
    weighted_sentences[sentence] = weight

# 가중치가 높은 문장 정렬
sorted_sentences = sorted(weighted_sentences.items(), key=lambda x: x[1], reverse=True)

# 4. 결과 출력
print("\n가중치 높은 문장 (상위 3개):")
for sentence, weight in sorted_sentences[:3]:
    print(f"문장: {sentence} | 가중치: {weight}")


TF-IDF 기반 키워드: ['의대는', '서울대', '만점자는', '가채점', '고려대', '수능', '것으로']

가중치 높은 문장 (상위 3개):
문장: 2025학년도 대학수학능력시험(수능) 가채점 기준 고득점자들의 윤곽이 드러나면서 지난해 1명뿐이던 수능 만점자가 이번에는 대폭 늘어날 것으로 전망된다 | 가중치: 2.5798241627000014
문장: 27일 교육계에 따르면 수능 가채점 결과 서울 서초구 세화고에서 3학년 A군과 같은 학교 졸업생 B씨가 각각 만점을 받은 것으로 파악됐다 | 가중치: 2.5798241627000014
문장: 장지환 서울중등진학지도연구회 교사는 “작년 수능보다 다소 쉬워 수도권 의대는 2∼3개 정도를 틀려야 합격권에 들 수 있을 것으로 추정된다”고 말했다 | 가중치: 2.5214146125762946


In [55]:
def prepare_text_for_kobart(weighted_sentences, top_n=3):
    """
    가중치 높은 문장들을 KoBART에 입력 가능한 텍스트로 정리하는 함수
    Args:
        weighted_sentences (dict): 문장별 가중치 딕셔너리
        top_n (int): 선택할 상위 문장 수
    Returns:
        str: KoBART에 입력 가능한 정리된 텍스트
    """
    # 1. 가중치 높은 문장 정렬 및 선택
    sorted_sentences = sorted(weighted_sentences.items(), key=lambda x: x[1], reverse=True)
    top_sentences = [sentence for sentence, _ in sorted_sentences[:top_n]]

    # 2. 문장 정리 및 중복 제거
    refined_sentences = []
    for sentence in top_sentences:
        # 중복 공백 제거
        sentence = " ".join(sentence.split())
        # 불필요한 문장 부호 제거
        for symbol in ["△", "●", "■", "…", "-", "※"]:
            sentence = sentence.replace(symbol, "")
        # 중복 제거
        if sentence not in refined_sentences:
            refined_sentences.append(sentence)

    # 3. 문장들을 하나로 연결
    result_text = " ".join(refined_sentences).strip()
    return result_text


# KoBART 입력용 텍스트 생성
kobart_input_text = prepare_text_for_kobart(weighted_sentences, top_n=3)

# 결과 출력
print("KoBART 입력 텍스트:")
print(kobart_input_text)


KoBART 입력 텍스트:
2025학년도 대학수학능력시험(수능) 가채점 기준 고득점자들의 윤곽이 드러나면서 지난해 1명뿐이던 수능 만점자가 이번에는 대폭 늘어날 것으로 전망된다 27일 교육계에 따르면 수능 가채점 결과 서울 서초구 세화고에서 3학년 A군과 같은 학교 졸업생 B씨가 각각 만점을 받은 것으로 파악됐다 장지환 서울중등진학지도연구회 교사는 “작년 수능보다 다소 쉬워 수도권 의대는 2∼3개 정도를 틀려야 합격권에 들 수 있을 것으로 추정된다”고 말했다


In [59]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Sentence-BERT 모델 로드
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

# 코사인 유사도 계산 함수
def calculate_contextual_similarity(summary1, summary2):
    """
    두 텍스트 간의 문맥 기반 코사인 유사도를 계산
    Args:
        summary1 (str): 첫 번째 요약문
        summary2 (str): 두 번째 요약문
    Returns:
        float: 코사인 유사도 점수
    """
    embeddings = model.encode([summary1, summary2])
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])
    return similarity[0][0]

# 1. 가중치 부여 요약문 입력
weighted_summary = """
27일 교육계에 따르면 2025학년도 대학수학능력시험(수능) 가채점 기준 고득점자들의 윤곽이 드러나면서 지난해 1명뿐이던 수능 만점자가 이번에는 대폭 늘어날 것으로 전망되어, 올해 1/5 만점자 역시 지난해보다 많을 것으로 보인다

"""
# 2. 가중치 미부여 요약문 입력
unweighted_summary = """
2025학년도 대학수학능력시험 원점수(300점 만점) 기준 고득점자들의 윤곽이 드러나면서 지난해 1명뿐이던 수능 만점자가 이번에는 대폭 늘어날 것으로 전망된다. 27일 교육계에 따르면 수능 가채점 결과 서울 서초구 세화고에서 3학년 A군과 같은 학교 졸업생 B씨가 각각 만점을 받은 것으로 파악됐다.
"""

# 3. GPT 요약문 입력
gpt_summary = """
2025학년도 수능 가채점 결과 만점자가 작년 1명에서 올해 약 9명으로 크게 증가할 것으로 보인다. 세화고 3학년 A군과 졸업생 B씨가 만점을 기록했으며, 수능이 작년보다 쉽게 출제돼 초고득점자가 급증했다. 서울대·연세대 의대는 원점수 297점, 수도권 의대는 293점이 합격권으로 예상되며, 과목 선택에 따라 표준점수에서 유불리가 발생할 수 있다.

"""

# 코사인 유사도 계산
cosine_weighted_gpt = calculate_contextual_similarity(weighted_summary, gpt_summary)
cosine_unweighted_gpt = calculate_contextual_similarity(unweighted_summary, gpt_summary)

# 결과 출력
print(f"가중치 부여 요약문과 GPT 요약문의 코사인 유사도: {cosine_weighted_gpt:.4f}")
print(f"가중치 미부여 요약문과 GPT 요약문의 코사인 유사도: {cosine_unweighted_gpt:.4f}")


C:\Users\kdoky\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


가중치 부여 요약문과 GPT 요약문의 코사인 유사도: 0.9667
가중치 미부여 요약문과 GPT 요약문의 코사인 유사도: 0.9660
